# Tensorflow Low Level API introduction

This notebook introduces the scala_tensorflow library low-level API for a simple regression model case.

Summary:

- load data into a Tensor
- get data back from Tensor to Scala Collections for visualization
- Create a model: Variables, and Placeholders, operations
- Training elements: Loss and Optimizer
- Graph and session
- Tensorboard vizualization
- Model Metrics


## Load tensorflow_scala dependency 

For the `scala_tensorflow` source, see:

https://github.com/eaplatanios/tensorflow_scala 

In [1]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module("org.platanios", "tensorflow_2.12"),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes("", "darwin-cpu-x86_64")
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")


A few imports to access the tensorflow_scala API

In [2]:
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap

import java.nio.file.Paths
import scala.util.Random

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.learn._

import org.platanios.tensorflow.api.learn.layers._

import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator

import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.core.client.FeedMap


import java.nio.file.Paths

import scala.util.Random

In [3]:
import sys.process._

import sys.process._

### Load a simple dataset

Machine learning is a data-driven process, let's load some simple data.

This is a csv with 2 columns, `Price` and `Surface` as double values.

We will want to get a model to predict land `Price` from `Surface`, a linear model should make the cut!

In [4]:
val rootDir = sys.env("HOME") + "/data/immo/"

rootDir: String = "/Users/xavier/data/immo/"

In [5]:
s"head ${rootDir}land-price.csv"!

Price,Surface
54,614
95,2900
155,2070
110,1098
95,1000
135,1150
92,2060
55,569
90,2000


res4: Int = 0

#### CSV parsing to Tensors, in pure Scala

We read the file line by line

Skip the header line

Split each row and convert to Double

Then we create a Tensor of Rank 1 (1-d array with 2 elements) for each row:

In [6]:
val bufferedSource = scala.io.Source.fromFile(s"${rootDir}land-price.csv")
lazy val tt = bufferedSource.getLines.drop(1).toVector
                       .map(_.split(",").map(_.trim.toFloat))
                       .map(arr => Tensor(arr(0), arr(1)))

bufferedSource: scala.io.BufferedSource = <iterator>
tt: Vector[Tensor[Float]] = <lazy>

We then add one dimension to the Tensor by collecting all records in a new one:

In [7]:
val data = Tensor(tt: _*)

data: Tensor[Float] = Tensor[Float, [19, 2]]

Note the shape, 19 elements of 2 values:

In [8]:
data.shape

res7: Shape = [19, 2]

Here are a few examples of loading Tensor values back into scala:

In [9]:
(data(0,0).scalar, data(0,1).scalar)

res8: (Float, Float) = (54.0F, 614.0F)

In [10]:
data(0).entriesIterator.toSeq

res9: Seq[Float] = Stream(54.0F, 614.0F)

In [11]:
data.entriesIterator.toSeq

res10: Seq[Float] = Stream(
  54.0F,
  614.0F,
  95.0F,
  2900.0F,
  155.0F,
  2070.0F,
  110.0F,
  1098.0F,
  95.0F,
  1000.0F,
  135.0F,
  1150.0F,
  92.0F,
  2060.0F,
  55.0F,
  569.0F,
  90.0F,
  2000.0F,
  109.0F,
  1100.0F,
  99.0F,
  1056.0F,
  61.5F,
  996.0F,
  129.0F,
  1336.0F,
  85.0F,
  574.0F,
  140.0F,
  1133.0F,
  150.0F,
  1568.0F,
  95.0F,
  750.0F,
  99.5F,
  1200.0F,
  85.0F,
  1060.0F
)

## Visualize data

In [12]:
import $ivy.`org.plotly-scala::plotly-almond:0.5.2`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

In [13]:
lazy val trace2 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  data(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Markers)
)
lazy val dataPlt = Seq(trace2)

lazy val layout = Layout(
  title = "Scatter Plot"
)



trace2: Scatter = <lazy>
dataPlt: Seq[Scatter] = <lazy>
layout: Layout = <lazy>

In [14]:
plot(dataPlt, layout)

res13: String = "plot-450095997"

## Build a linear model:


In [15]:
val inputs      = tf.placeholder[Float](Shape(-1, 1))
val outputs     = tf.placeholder[Float](Shape(-1, 1))

inputs: Output[Float] = Output(Placeholder, 0)
outputs: Output[Float] = Output(Placeholder_1, 0)

In [16]:
val (predictions, weights) = tf.nameScope("Linear") {
  val weights = tf.variable[Float]("weights", Shape(1, 1), tf.ZerosInitializer)
  (tf.matmul(inputs, weights), weights)
}

predictions: Output[Float] = Output(Linear/MatMul, 0)
weights: Variable[Float] = Variable(
  DataType("Float", 1, Some(4), DT_FLOAT),
  Output(weights, 0),
  weights/InitializationAssign,
  null,
  Output(weights/Read/ReadVariable, 0)
)

In [17]:
val loss        = tf.sum(tf.square(predictions - outputs))
val optimizer   = tf.train.AdaGrad(1.0f)
val trainOp     = optimizer.minimize(loss)

loss: Output[Float] = Output(Sum, 0)
optimizer: ops.training.optimizers.AdaGrad = org.platanios.tensorflow.api.ops.training.optimizers.AdaGrad@ab54147
trainOp: UntypedOp = Minimize/Finish

In [18]:
val sess = Session()

sess: Session = org.platanios.tensorflow.api.core.client.Session@2ee9ee3c

In [19]:
sess.run(targets = tf.globalVariablesInitializer())

res18: implicits.helpers.OutputToTensor.<refinement>.this.type.V = List()

In [20]:
val inTensor = data(---, 1).expandDims(-1)
val outTensor = data(---, 0).expandDims(-1)

inTensor: Tensor[Float] = Tensor[Float, [19, 1]]
outTensor: Tensor[Float] = Tensor[Float, [19, 1]]

In [21]:
val feedMap = Seq(
    FeedMap(inputs, inTensor),
    FeedMap(outputs, outTensor)
    )

feedMap: Seq[FeedMap] = List(
  org.platanios.tensorflow.api.core.client.FeedMap@14463f10,
  org.platanios.tensorflow.api.core.client.FeedMap@50953bce
)

In [32]:
val (currentLoss, currentWeights) = sess.run(fetches = (loss, weights.value), 
                                              targets = Set(trainOp), feeds = feedMap)

currentLoss: Tensor[Float] = Tensor[Float, []]
currentWeights: Tensor[Float] = Tensor[Float, [1, 1]]

In [33]:
currentWeights.entriesIterator.toSeq

res32: Seq[Float] = Stream(0.06895473F)

In [34]:
sess.graph.toMetaGraphDef()

res33: org.tensorflow.framework.MetaGraphDef = meta_info_def {
  tensorflow_version: "1.12.0-rc0"
}
graph_def {
  node {
    name: "Placeholder"
    op: "Placeholder"
    attr {
      key: "dtype"
      value {
        type: DT_FLOAT
      }
    }
    attr {
      key: "shape"
      value {
        shape {
          dim {
            size: -1
          }
          dim {
            size: 1
          }
        }
      }
    }
  }
  node {
    name: "Placeholder_1"
    op: "Placeholder"
    attr {
      key: "dtype"
      value {
        type: DT_FLOAT
      }
    }
    attr {
      key: "shape"
      value {
...

In [ ]:
val path = java.nio.file.Paths.get("/tmp/000tfsc")
tf.summary.FileWriter(path, sess.graph)


In [35]:
val preds = sess.run(fetches = predictions, feeds = FeedMap(inputs -> inTensor))

preds: Tensor[Float] = Tensor[Float, [19, 1]]

In [36]:
preds(---, 0).entriesIterator.toSeq.zip(data(---, 0).entriesIterator.toSeq)

res35: Seq[(Float, Float)] = Stream(
  (42.324295F, 54.0F),
  (199.90303F, 95.0F),
  (142.6894F, 155.0F),
  (75.68742F, 110.0F),
  (68.932076F, 95.0F),
  (79.27189F, 135.0F),
  (142.00008F, 92.0F),
  (39.22235F, 55.0F),
  (137.86415F, 90.0F),
  (75.82529F, 109.0F),
  (72.792274F, 99.0F),
  (68.65635F, 61.5F),
  (92.093254F, 129.0F),
  (39.567013F, 85.0F),
  (78.100044F, 140.0F),
  (108.0855F, 150.0F),
  (51.69906F, 95.0F),
  (82.7185F, 99.5F),
  (73.068F, 85.0F)
)

## Plot Predictions vs Actual

In [37]:
lazy val trace2 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  data(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Markers)
)

lazy val trace1 = Scatter(
  data(---, 1).entriesIterator.toSeq,
  preds(---, 0).entriesIterator.toSeq,
  mode = ScatterMode(ScatterMode.Lines)
)

lazy val dataPlt = Seq(trace1, trace2)

lazy val layout = Layout(
  title = "Scatter Plot"
)


trace2: Scatter = <lazy>
trace1: Scatter = <lazy>
dataPlt: Seq[Scatter] = <lazy>
layout: Layout = <lazy>

In [38]:
plot(dataPlt, layout)

res37: String = "plot-1816658790"